# 1. Initialization & Reading of the Dataset

#### Downloading the ETL9 Dataset and enclosing it into one npz file. This file will be used to read and train the dataset. 

In [1]:
import struct
from PIL import Image
import numpy as np

def read_record_ETL9(f):
    s = f.read(8199)
    r = struct.unpack('>2H8sI4B4H2B30x8128s11x', s)
    iF = Image.frombytes('F', (128, 127), r[14], 'bit', 4)
    iL = iF.convert('L')
    return r + (iL,)

def read_kanji():
    kanji = np.zeros([883, 160, 127, 128], dtype=np.uint8)
    for i in range(1, 33):
        filename = 'ETL8G/ETL9_{:02d}'.format(i)
        with open(filename, 'rb') as f:
            for dataset in range(5):
                char = 0
                for j in range(956):
                    r = read_record_ETL8G(f)
                    if not (b'.HIRA' in r[2] or b'.WO.' in r[2]): 
                        kanji[char, (i - 1) * 5 + dataset] = np.array(r[-1])
                        char += 1
    np.savez_compressed("kanji.npz", kanji)

read_kanji()

# 2. Modifying the Dataset

#### Splitting the dataset into 4 parts:
- training images
- training labels
- testing images
- testing labels


In [2]:
import skimage.transform
import numpy as np
from sklearn.model_selection import train_test_split

kanji = 879
rows = 64
cols = 64

kan = np.load("kanji.npz")['arr_0'].reshape([-1, 127, 128]).astype(np.float32)

kan = kan/np.max(kan)

train_images = np.zeros([kanji * 160, rows, cols], dtype=np.float32)

arr = np.arange(kanji)
train_labels = np.repeat(arr, 160)

for i in range( (kanji+4) * 160):
        if int(i/160) != 88 and int(i/160) != 219 and int(i/160) != 349 and int(i/160) != 457:
            if int(i/160) < 88:
                train_images[i] = skimage.transform.resize(kan[i], (rows, cols))
            if int(i/160) > 88 and int(i/160) < 219:
                train_images[i-160] = skimage.transform.resize(kan[i], (rows, cols))
            if int(i/160) > 219 and int(i/160) < 349:
                train_images[i-320] = skimage.transform.resize(kan[i], (rows, cols))
            if int(i/160) > 349 and int(i/160) < 457:
                if int(i/160) > 457:
                    train_images[i-640] = skimage.transform.resize(kan[i], (rows, cols))
      
train_images, test_images, train_labels, test_labels = train_test_split(train_images, train_labels, test_size=0.2)

np.savez_compressed("kanji_train_images.npz", train_images)
np.savez_compressed("kanji_train_labels.npz", train_labels)
np.savez_compressed("kanji_test_images.npz", test_images)
np.savez_compressed("kanji_test_labels.npz", test_labels)

# 3. Creating the Model 

#### Here we begin loading in the training and test files and start to train the model. 
#### The process will take about 12 hours. (Set callbacks.EarlyStopping so that best epoch can be reached without continuing the training).
#### Once finished, a .h5 file will be created.

In [3]:
from tensorflow import keras
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

train_images = np.load("kanji_train_images.npz")['arr_0']
train_labels = np.load("kanji_train_labels.npz")['arr_0']
test_images = np.load("kanji_test_images.npz")['arr_0']
test_labels = np.load("kanji_test_labels.npz")['arr_0']

if K.image_data_format() == "channels_first":
  train_images = train_images.reshape(train_images.shape[0], 1,64,64)
  test_images = test_images.reshape(test_images.shape[0], 1,64,64)
  shape = (1,64,64)
else:
  train_images = train_images.reshape(train_images.shape[0], 64, 64, 1)
  test_images = test_images.reshape(test_images.shape[0], 64, 64, 1)
  shape = (64,64,1)
  
datagen = ImageDataGenerator(rotation_range=15,zoom_range=0.2)
datagen.fit(train_images)
model = keras.Sequential([
  keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=shape),
  keras.layers.MaxPooling2D(2,2),
  keras.layers.Conv2D(64, (3,3), activation='relu'),
  keras.layers.MaxPooling2D(2,2),
  keras.layers.Flatten(),
  keras.layers.Dropout(0.5),
  keras.layers.Dense(2048, activation='relu'),
  keras.layers.Dense(879, activation="softmax")
])

model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])
              
model.fit_generator(datagen.flow(train_images,train_labels,shuffle=True),epochs=50,validation_data=(test_images,test_labels),callbacks = [keras.callbacks.EarlyStopping(patience=8,verbose=1,restore_best_weights=True),keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=3,verbose=1)])

model.save("kanji.h5")

Using TensorFlow backend.
/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
3516/3516 [==============================] - 1295s 368ms/step - loss: 5.2747 - accuracy: 0.1267 - val_loss: 3.9380 - val_accuracy: 0.3558
Epoch 2/50
3516/3516 [==============================] - 1297s 369ms/step - loss: 4.0428 - accuracy: 0.3291 - val_loss: 3.8709 - val_accuracy: 0.3749
Epoch 3/50
3516/3516 [==============================] - 1298s 369ms/step - loss: 3.9608 - accuracy: 0.3494 - val_loss: 3.8532 - val_accuracy: 0.3787
Epoch 4/50
3516/3516 [==============================] - 1296s 369ms/step - loss: 3.9139 - accuracy: 0.3613 - val_loss: 3.8451 - val_accuracy: 0.3824
Epoch 5/50
3516/3516 [==============================] - 1300s 370ms/step - loss: 3.8982 - accuracy: 0.3663 - val_loss: 3.8335 - val_accuracy: 0.3848
Epoch 6/50
3516/3516 [==============================] - 1300s 370ms/step - loss: 3.9001 - accuracy: 0.3689 - val_loss: 3.8330 - val_accuracy: 0.3857
Epoch 7/50
3516/3516 [==============================] - 1293s 368ms/step - loss: 3.8872 - accuracy: 0.3717

# 4. Convert to Tensorflow Lite

#### Now that the .h5 model has been created, we need to convert it to a format that is better suited for small devices such as a smartphone.
#### We will convert the .h5 model to .tflite file so it can be used in an Android Application for a visual output. 

In [5]:
import tensorflow as tf

converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model_file("kanji2.h5")

converter.experimental_new_converter = True

tflite_model = converter.convert()

open("kanji.tflite", "wb").write(tflite_model)



/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


INFO:tensorflow:Assets written to: /tmp/tmpowwinwe1/assets


INFO:tensorflow:Assets written to: /tmp/tmpowwinwe1/assets


INFO:tensorflow:Restoring parameters from /tmp/tmpowwinwe1/variables/variables


INFO:tensorflow:Restoring parameters from /tmp/tmpowwinwe1/variables/variables


...

INFO:tensorflow: tensor name: StatefulPartitionedCall:0, shape: (-1, 879), type: DT_FLOAT


INFO:tensorflow: tensor name: StatefulPartitionedCall:0, shape: (-1, 879), type: DT_FLOAT


INFO:tensorflow:Restoring parameters from /tmp/tmpowwinwe1/variables/variables


INFO:tensorflow:Restoring parameters from /tmp/tmpowwinwe1/variables/variables


110126300